# _LOGISTIC REGRESSION ALGORITHM_

Logistic Regression is a _Binary linear Classifier_ which is a function of the form: $f(x) = sign(x^Tw + w_0)$, where $w \in \mathbb{R}^d$ and $w_0 \in \mathbb{R}$

The pair _$(w,w_0)$_ define an `affine hyperplane` for which it is important to develop the right geometric undestanding in relation with the linear classifier.

## AFFINE HYPERPLANE:
A hyperplane `H` can be represented by a vector **w** as follows:

$H = \{x \in \mathbb{R}^d | x^Tw = 0\}$

An `affine hyperplane` is a hyperplane translated using the scalar $w_0$, so we can think of a affine hyperplane as:

$H = x^Tw + w_0 = 0$

Where the hyperplane has been shifted by a distance $w_0 / \|w\|_2$ in the direction $w$, then for a given $w, w_0$ and an input _**x**_ the inequlity $x^Tw + w_0 > 0$ says that _**x**_ is on the far side of an affine hyperplane H in the direction $w$ points.


### _Affine Hyperplane_
![Hyperplane](.\image\hyperplane.png)

Also, _Logistic Regression_ can be combined to the separating hyperplane idea to probability thru a _Bayse Classifier_.

A classifier $f$ with property: $f(x) = arg max_{y\in Y}  P(Y=y | X=x)$ for $x \in X$ is called the `Bayes Classifier`.

From **Bayes Rule** we equivalently have $f(x) = arg max_{y\in Y}  P(Y=y)P(X=x | Y=y)$. Where:
- $P(Y=y)$ is called the _class prior_ and $P(Y=y) = \pi_y, y \in \{-1,1\}$
- $P(X=x | Y=y)$ is called the _class conditional distribution_ of $X$

If $X$ is a continuous-valued random variable $P(X=x | Y=y)$ can be replaced by the _class conditional density_ $p(x|Y=y)$.

In the binary case, we declare $y=1$ if: $p(x|y=1)P(Y=1) > p(x|y=-1)P(Y=-1) \iff ln  p(x|y=1)p(y=1)/p(x|y=-1)p(y=-1) > 0$ 

where: $y \sim Benoulli(\pi)$ and $x|y \sim ( \mu_y,\sum )$

The second line is refered as the `log odds`.


## LOG ODDS AND HYPERPLANES:
Classifying $x$ with the `log odds`

$\large L = ln \frac{p(y = +1|x)}{p(y = -1|x)}$

We notice that:
- $L \gg 0$: more confident $y = +1$
- $L \ll 0$: more confident $y = -1$
- $L = 0$: can go either way

The linear function $x^Tw + w_0$ captures these three objectives, so we can plug in the hyperplane representation for the `log odds`.

$\large L = ln \frac{p(y = +1|x)}{p(y = -1|x)} = x^Tw + w_0$

Setting $p(y = -1|x) = 1 - p(y = +1|x)$ and solving for $p(y = +1|x)$:

$\Large p(y = +1|x) = \frac{exp^{x^Tw + w_0}}{1 + exp^{x^Tw + w_0}} = \sigma (x^Tw + w_0)$

This is called _sigmoid function_


### _Sigmoid Function_
![Sigmoid](.\image\sigmoid.png)

The _sigmoid Function_ $\sigma (x^Tw + w_0)$ maps $x$ to $p(y = +1|x)$.

The function captures our desire to be **more confident** as we move away from the separating hyperplane, defined by the _x-axis_.

## MODEL REPRESENTATION:
Let $(x_1,y_1), ..., (x_n,y_n)$ be a set of binary labeled data with $y \in \{-1,+1\}$.

_**Logistic Regression**_ models each $y_i$ as independently generated, with $P(y = +1|x_i,w) = \sigma (x_i^Tw)$

This is a discriminative classifier because $x$ is not directly modeled so, the joint likelihood of $y_1,....,y_n$ can be written as:

$\large p(y_1,...,y_n|x_1,...,x_n,w) = \prod_{i=1}^{n} \sigma_i (x_i^Tw)$

And we want to minimize over _$\large w$_

$\large w = arg max_w \sum_{i=1}^{n} ln \sigma_i (x_i^Tw)$

we can’t directly set $\nabla_w \sigma_i (x_i^Tw) = 0$, however $\nabla_w \sigma_i (x_i^Tw)$ does tell us the **direction** in which the function is _increasing_ with $w$ and therefore we can update the model parameters thru an iterative algorithm.

This is a very general method for optimizing an objective function called `gradient descent`.


# GRADIENT DESCENT:

`Gradient descent` is an optimization algorithm used to find the values of parameters of a function $f$ that minimizes a cost function. `Gradient descent` is best used when the parameters cannot be calculated analytically and must be searched for by an optimization algorithm.

`Gradient descent` can be slow to run on very large datasets. Because one iteration of the gradient descent algorithm requires a prediction for each instance in the training dataset, it can take a long time when you have many instances. In this situations we can use a variation of gradient descent called `stochastic gradient descent`, where the same procedure is run but the update to the parameters is performed for each training instance, rather than at the end of the batch of all instances.


## LOGISTIC REGRESSION BY STOCHASTIC GRADIENT DESCENT:
We can apply `stochastic gradient descent` to the problem of finding the coefficients for the logistic regression
model.

We have training data $(x_1,y_i), ..., (x_n,y_n)$ and a step size $\large \eta > 0$

Let's start off by assigning 0 to each coefficient:

- Set $w^{(1)} = \vec{\mathbf{0}}$

Then calculate the new coefficient values using a simple update equation:

- For step $t = 1,2,....$ update

$$\large w^{t+1} = w^t + \eta * (y - y^-) * (1 - \sigma(x^Tw))* y^-x$$

Repeat the process and update the model for each training instance in the dataset. A single iteration through the training dataset is called an epoch. It is common to repeat the stochastic gradient descent procedure for a fixed number of epochs.

Ok, let's start with the implementation of `Logistic Regression` from scratch.

In [1]:
##IMPORTING ALL NECESSARY SUPPORT LIBRARIES
from math import exp
from math import inf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
def prepare_data(dataset):
    '''
    arguments:
        > dataset: 2-D Array where last column is the target
    returns:
        > data: Augmented 2-D Array with 1 + features-columns
        > labels: 1-D Array with the target values
        > weights: 1-D Array with model coefficients initialized
    '''
    
    data = []
    labels = []
    for row in dataset:
        labels.append(row.pop())
        row.insert(0,1)
        data.append([x for x in row])
        
    weights = [0 for i in range(len(data[0]))]
        
    return data,labels,weights

In [3]:
def predict(X, weights):
    pred = sum([x*w for x,w in zip(X,weights)])
    e = exp(-pred)
    if e == inf:
        return 1
    else:
        return 1.0/(1.0 + e)

In [4]:
def update_weights(X_train,y_train,weights,*,eta=0.1,epochs=100):
    '''
    arguments:
        > X_train: Augmented 2-D Array with 1 + features-columns
        > y_train: 1-D Array with the target values
        > weights: 1-D Array with model coefficients
        > eta: Learning Rate
        > epochs: Number of max iterations
    returns:
        > weights: 1-D Array with model coefficients updated
    '''
    for epoch in range(epochs):
        SSE = 0
        
        for idx, row in enumerate(X_train):
            pred = predict(row, weights)
            error = y_train[idx] - pred
            SSE += error**2
            
            for i in range(len(weights)):
                weights[i] = weights[i] + eta * error * (1 - pred) * pred * row[i]
                
            #print('epoch: {0}, SSE: {1}'.format(epoch,SSE))
            
    return weights


In order to verify proper implementation a **toy dataset** is used to evaluate the algorithm.

In [5]:
dataset = [[2.7810836,2.550537003,0],
[1.465489372,2.362125076,0],
[3.396561688,4.400293529,0],
[1.38807019,1.850220317,0],
[3.06407232,3.005305973,0],
[7.627531214,2.759262235,1],
[5.332441248,2.088626775,1],
[6.922596716,1.77106367,1],
[8.675418651,-0.242068655,1],
[7.673756466,3.508563011,1]]

#OUR DATASET IS THEN PRE-PROCESSED IN ORDER TO BE READY FOR OUR 'LOGISTIC REGRESSION MODEL'
x_data, labels, weights = prepare_data(dataset)

Once we have our data ready, the coefficients have to be _learned_ to run our predictions. We will run our `update_weights` function with a learning rate $\eta = 0.3$ and a total iterations on our data equal to $100$ times

In [6]:
coeff = update_weights(x_data,labels,weights,eta=0.3,epochs=100)
coeff

[-0.8596443546618896, 1.5223825112460005, -2.218700210565016]

With our new `coefficients` we can test how well our model can predict the target value

In [7]:
for idx, row in enumerate(x_data):
    yhat = predict(row, coeff)
    print("Expected={0}, Predicted={1}".format(labels[idx], yhat))

Expected=0, Predicted=0.09240238749672108
Expected=0, Predicted=0.020443070793394046
Expected=0, Predicted=0.004270645838622242
Expected=0, Predicted=0.05460100428754744
Expected=0, Predicted=0.05402203604622585
Expected=1, Predicted=0.9903433037486284
Expected=1, Predicted=0.932411366865848
Expected=1, Predicted=0.9968264836375801
Expected=1, Predicted=0.9999974635411156
Expected=1, Predicted=0.9542746551950381


# _LOGISTIC REGRESSION APPLICATION_

From the `UCI Machine Learning Repository` which contains data on female patients at least 21 years old of Pima Indian heritage, we will train our `logistic regression` model using stochastic gradient descent on the **pima-indians-diabetes** dataset.

The dataset have 768 instances and the following 8 attributes:
- Number of times pregnant (preg)
- Plasma glucose concentration a 2 hours in an oral glucose tolerance test (plas)
- Diastolic blood pressure in mm Hg (pres)
- Triceps skin fold thickness in mm (skin)
- 2-Hour serum insulin in mu U/ml (insu)
- Body mass index measured as weight in kg/(height in m)^2 (mass)
- Age in years (age)
- Diabetes pedigree function (pedi)

A particularly interesting attribute used in the study was the Diabetes Pedigree Function, pedi. It provided some data on diabetes mellitus history in relatives and the genetic relationship of those relatives to the patient. This measure of genetic influence gave us an idea of the hereditary risk one might have with the onset of diabetes mellitus


To compare the performance of our _Classifier_ on the **pima-indians-diabetes** dataset, a Logistic Regression model from `sklearn` will be fit on the dataset and classification report for both models is generated.

In [8]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

In [9]:
##LOADING 'PIMA INDIANS DIABETES' DATASET
col = ['Preg', 'Plas', 'Pres', 'Skin', 'Insu', 'Mass', 'Pedi', 'Age', 'target']
Pima = pd.read_csv('./data/pima-indians-diabetes.csv', names = col)
Pima.head()

,Preg,Plas,Pres,Skin,Insu,Mass,Pedi,Age,target
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [10]:
Pima.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
Preg      768 non-null int64
Plas      768 non-null int64
Pres      768 non-null int64
Skin      768 non-null int64
Insu      768 non-null int64
Mass      768 non-null float64
Pedi      768 non-null float64
Age       768 non-null int64
target    768 non-null int64
dtypes: float64(2), int64(7)
memory usage: 54.1 KB


The dataset is first split into `feature values` and `labels` array loaded, and each feature-column is normalized to values in the range of 0 to 1.
Once the preprocessing is complete the dataset will be split into a `Training` & `Test` dataset.

In [11]:
X_raw, y, weights = prepare_data(Pima.values.tolist())
scaler = MinMaxScaler()
X_norm = scaler.fit_transform(X_raw)
X_train, X_test, y_train, y_test = train_test_split(X_norm, y, test_size=0.30, random_state=5)

Once we have our data ready, the coefficients have to be _learned_ to run our predictions. We will run our `update_weights` function with a learning rate $\eta = 0.05$ and a total iterations on our data equal to $10000$ times

In [12]:
logr_coef = update_weights(X_train,y_train,weights,eta=0.05,epochs=10000)
print(logr_coef)

[0.0, 1.3154202563989514, 2.2690659261588677, -5.019016039713838, -0.3123259665800087, 0.7656450106312648, 0.017726651509694727, 1.564383609909205, 2.372423223321087]


In [14]:
##CREATE CUSTOMIZED PREDICTIONS ARRAY
pred = [predict(row,logr_coef) for row in X_test]
cust_pred = np.array([1 if x >=0.5 else 0 for x in pred])

Now an instance of `sklearn` _Logistic Regression_ model is created and fit it with the training data and an array of predictions is obtained in order to get out performance comparation

In [15]:
##GET AND INSTANCIATE OF LOGISTIC REGRESSION MODEL
sk_lr = LogisticRegression()
sk_lr.fit(X_train, y_train)

##CREATE SKLEARN PREDICTIONS ARRAY
sk_pred = sk_lr.predict(X_test)

E:\devop\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [16]:
sk_lr.coef_

array([[ 0.        ,  1.59953863,  3.64731438, -1.14311614, -0.15296794,
         0.39461853,  2.05479615,  1.41757151,  1.24727379]])

By last, a comparison on both models is performed thru a _Classification Report_

In [17]:
print("Sklearn:")
print(classification_report(y_test, sk_pred))
print("Custom:")
print(classification_report(y_test, cust_pred))

Sklearn:
              precision    recall  f1-score   support

         0.0       0.82      0.84      0.83       160
         1.0       0.62      0.59      0.60        71

   micro avg       0.76      0.76      0.76       231
   macro avg       0.72      0.71      0.72       231
weighted avg       0.76      0.76      0.76       231

Custom:
              precision    recall  f1-score   support

         0.0       0.79      0.78      0.78       160
         1.0       0.51      0.52      0.52        71

   micro avg       0.70      0.70      0.70       231
   macro avg       0.65      0.65      0.65       231
weighted avg       0.70      0.70      0.70       231

